# overriding the header parameters and remove the rotation from affine matrix

* remove the rotation from the header and only keep scaling and translation

## 1. import packages

In [2]:
import nibabel as nib
import numpy as np
import os
import tabulate

## 2. load file explore

In [5]:
base_dir = r"../"
file_name = r"sub-HC001_ses-01_acq-mp2rage_T1map.nii.gz"
file_path = os.path.join(base_dir, file_name)
file_path

'../sub-HC001_ses-01_acq-mp2rage_T1map.nii.gz'

In [6]:
nib_file = nib.load(file_path)

In [7]:
header = nib_file.header
print(header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 240 320 320   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.  0.8 0.8 0.8 0.  0.  0.  0. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'6.0.0'
aux_file        : b''
qform_code      : scanner
sform_code      : unknown
quatern_b       : -0.06856307
quatern_c       : -0.015582562
quatern_d       : -0.019387364
qoffset_x       : -92.05724
qoffset_y       : -107.88861
qoffset_z       : -116.

## 3. extract voxel spacing and qoffsets

In [8]:
voxel_spacing = header.get_zooms()
voxel_spacing

(0.8, 0.8, 0.8)

In [9]:
qoffset_x = header['qoffset_x']
qoffset_y = header['qoffset_y']
qoffset_z = header['qoffset_z']

print(qoffset_x, qoffset_y, qoffset_z)

-92.05724 -107.88861 -116.848816


## 4. create new affine

In [10]:
new_affine = np.array([
    [voxel_spacing[0], 0, 0, qoffset_x],
    [0, voxel_spacing[1], 0, qoffset_y],
    [0, 0, voxel_spacing[2], qoffset_z],
    [0, 0, 0, 1]
])

new_affine

array([[   0.80000001,    0.        ,    0.        ,  -92.05724335],
       [   0.        ,    0.80000001,    0.        , -107.88861084],
       [   0.        ,    0.        ,    0.80000001, -116.84881592],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

## 5. override srow_* in header

In [11]:
new_header = header.copy()

In [12]:
new_header['srow_x'] = new_affine[0,:]
new_header['srow_y'] = new_affine[1,:]
new_header['srow_z'] = new_affine[2,:]

In [13]:
print(new_header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 240 320 320   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.  0.8 0.8 0.8 0.  0.  0.  0. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'6.0.0'
aux_file        : b''
qform_code      : scanner
sform_code      : unknown
quatern_b       : -0.06856307
quatern_c       : -0.015582562
quatern_d       : -0.019387364
qoffset_x       : -92.05724
qoffset_y       : -107.88861
qoffset_z       : -116.

## 6. create new file using new affine and new header

In [14]:
updated_img = nib.Nifti1Image(nib_file.get_fdata(), new_affine, new_header)

In [15]:
nib.save(updated_img, os.path.join(base_dir, "updated_" + file_name))

## 7. load updated file and compare

In [16]:
updated_file = nib.load(os.path.join(base_dir, "updated_" + file_name))

In [17]:
header_original = nib_file.header.structarr
header_sampled = updated_file.header.structarr

header_params = nib_file.header.keys()

compare = []
for param in header_params:
    compare.append([param, header_original[param].tolist(), header_sampled[param].tolist()])

table = tabulate.tabulate(compare, tablefmt='html')

table

sizeof_hdr,348,348
data_type,b'',b''
db_name,b'',b''
extents,0,0
session_error,0,0
regular,b'r',b'r'
dim_info,0,0
dim,"[3, 240, 320, 320, 1, 1, 1, 1]","[3, 240, 320, 320, 1, 1, 1, 1]"
intent_p1,0.0,0.0
intent_p2,0.0,0.0
intent_p3,0.0,0.0


* the quartenian values are set to 0, which means no rotation